In [1]:
import numpy as np
import tensorflow as tf

from keras.layers import Dense, Activation
from keras.models import Model
from keras.applications import imagenet_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.activations import relu, sigmoid, softmax

from sklearn.metrics import confusion_matrix

import os
import itertools
import shutil
import random
import matplotlib.pyplot as plt

import import_ipynb
import mobileNet_preprocess

importing Jupyter notebook from mobileNet_preprocess.ipynb
Found 1662 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
Found 300 images belonging to 10 classes.


2022-06-27 19:44:53.169835: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

2022-06-27 19:44:57.465526: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2022-06-27 19:44:57.514851: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2022-06-27 19:44:57.532013: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 16056320 exceeds 10% of free system memory.
2022-06-27 19:44:57.543797: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 32112640 exceeds 10% of free system memory.
2022-06-27 19:44:57.556172: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 32688640 exceeds 10% of free system memory.


167/167 - 63s - loss: 0.6726 - accuracy: 0.7930 - val_loss: 0.4125 - val_accuracy: 0.8733 - 63s/epoch - 377ms/step
Epoch 2/10
167/167 - 59s - loss: 0.1228 - accuracy: 0.9807 - val_loss: 0.1250 - val_accuracy: 0.9600 - 59s/epoch - 355ms/step
Epoch 3/10
167/167 - 50s - loss: 0.0557 - accuracy: 0.9958 - val_loss: 0.0552 - val_accuracy: 0.9933 - 50s/epoch - 300ms/step
Epoch 4/10
167/167 - 63s - loss: 0.0342 - accuracy: 0.9976 - val_loss: 0.0610 - val_accuracy: 0.9767 - 63s/epoch - 375ms/step
Epoch 5/10
167/167 - 56s - loss: 0.0263 - accuracy: 0.9970 - val_loss: 0.0408 - val_accuracy: 0.9933 - 56s/epoch - 336ms/step
Epoch 6/10


In [30]:
mobile = tf.keras.applications.mobilenet.MobileNet()
mobile.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [25]:
#fetching the model and exclusding the last 6 layers of the mobile net
x = mobile.layers[-6].output
 #store it in a variable x
output = Dense(units=10, activation='softmax')(x)


In [26]:
#construct the reconstructed model
model = Model(inputs=mobile.input, outputs=output) #we have out original mobile net layer
# we have included our layer for the last 5/6 that we ommiteed, therefoer, we have to train 
#this new model, and this we do by, freezing some parts of the previous inorder to maintin 
#the weights

In [27]:
#experimantally came up with 26 inrder ro have a decently performing model
for layer in model.layers[:-23]: #choose the last 23, omit the first (88-23)
    layer.trainable = False #train 23rd to last layer
#model.summary()

In [28]:
#Compile the Model for Training

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy')

In [29]:
#Train the Model by calling the fit function
model.fit(
    x=train_batches,
    steps_per_epoch=len(train_batches),
    validation_data=valid_batches,
    validation_steps=len(valid_batches),
    epochs=30,
    verbose=2
    )

NameError: name 'train_batches' is not defined